# Introduction to generative models and variational inference

# Traditional probabilistic programming-based implementation of variational inference

## Inverse Gamma-Normal conjugate model from Turing.jl

This section is slightly edited for clarity from the [Turing.jl variational inference tutorial](https://github.com/TuringLang/TuringTutorials/blob/master/9_VariationalInference.ipynb).

### Import libraries

In [10]:
# ~1m50s initial
# ~1s subsequent
using Random
using Turing
using Turing: Variational

Random.seed!(42);

### Define model

The Normal-(Inverse)Gamma conjugate model is defined by a generative process

\begin{align}
    s &\sim \mathrm{InverseGamma}(2, 3) \\
    m &\sim \mathcal{N}(0, s) \\
    x_i &\overset{\text{i.i.d.}}{=} \mathcal{N}(m, s), \quad i = 1, \dots, n
\end{align}

Generate synthetic data samples. This is the key capability of generative models. This ability derives from the fact that the joint distribution over observed and latent variables is being considered.

In [11]:
x = randn(2000);

In [13]:
x[1:5]

5-element Array{Float64,1}:
 -0.5560268761463861
 -0.444383357109696
  0.027155338009193845
 -0.29948409035891055
  1.7778610980573246

Define the model as an instance of the type `Turing.Model`.

In [14]:
@model model(x) = begin
    s ~ InverseGamma(2,3)
    m ~ Normal(0.0, sqrt(s))
    for i = 1:length(x)
        x[i] ~ Normal(m, sqrt(s))
    end
end

model (generic function with 1 method)

In [28]:
print(@doc(@model))

```
@model(expr[, warn = true])
```

Macro to specify a probabilistic model.

If `warn` is `true`, a warning is displayed if internal variable names are used in the model definition.

# Examples

Model definition:

```julia
@model function model(x, y = 42)
    ...
end
```

To generate a `Model`, call `model(xvalue)` or `model(xvalue, yvalue)`.


Construct an instance of the model `m`.

In [15]:
m = model(x);

In [17]:
typeof(m)

DynamicPPL.Model{var"#6#7",(:x,),(),(),Tuple{Array{Float64,1}},Tuple{}}

In [18]:
m

DynamicPPL.Model{var"#6#7",(:x,),(),(),Tuple{Array{Float64,1}},Tuple{}}(:model, var"#6#7"(), (x = [-0.5560268761463861, -0.444383357109696, 0.027155338009193845, -0.29948409035891055, 1.7778610980573246, -1.14490153172882, -0.46860588216767457, 0.15614346264074028, -2.641991008076796, 1.0033099014594844  …  -1.429934217684174, 1.0569955382840672, 0.06237396220057982, -0.4018525193010452, 0.8502920979167766, -1.762950599025789, 1.3645385605509757, -1.152476312640822, -1.8572897801688275, 0.25976965508292527],), NamedTuple())

See [DynamicPPL.jl/src/model.jl](https://github.com/TuringLang/DynamicPPL.jl/blob/master/src/model.jl) for the definition of the `DynamicPPL.Model` type.

### Sample from the model with MCMC/HMC

Here we use the [no U-turn sampler (NUTS)](http://chi-feng.github.io/mcmc-demo/app.html) to generate samples from the model.

In [20]:
# ?2m12s
samples_nuts = sample(m, NUTS(200, 0.65), 10000);

┌ Info: Found initial step size
│   ϵ = 0.05
└ @ Turing.Inference /home/jovyan/.julia/packages/Turing/O1Pn0/src/inference/hmc.jl:195
Sampling: 100%|█████████████████████████████████████████| Time: 0:01:45


In [23]:
samples_nuts

Chains MCMC chain (10000×14×1 Array{Float64,3}):

Iterations        = 1:10000
Thinning interval = 1
Chains            = 1
Samples per chain = 10000
parameters        = m, s
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat 
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64 

           m   -0.0277    0.0226     0.0002    0.0002   9465.7308    1.0002
           s    1.0241    0.0324     0.0003    0.0003   9204.4318    0.9999

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           m   -0.0729   -0.0427   -0.0276   -0.0126    0.0163
           s    0.9627    1.0018    1.0233    1.0456    1.0891


### Sample from the model with VI

In [24]:
print(@doc(Variational.vi))

```
vi(model, alg::VariationalInference)
vi(model, alg::VariationalInference, q::VariationalPosterior)
vi(model, alg::VariationalInference, getq::Function, θ::AbstractArray)
```

Constructs the variational posterior from the `model` and performs the optimization following the configuration of the given `VariationalInference` instance.

# Arguments

  * `model`: `Turing.Model` or `Function` z ↦ log p(x, z) where `x` denotes the observations
  * `alg`: the VI algorithm used
  * `q`: a `VariationalPosterior` for which it is assumed a specialized implementation of the variational objective used exists.
  * `getq`: function taking parameters `θ` as input and returns a `VariationalPosterior`
  * `θ`: only required if `getq` is used, in which case it is the initial parameters for the variational posterior


`vi` takes 
1. the `Model` you want to approximate and
1. a `VariationalInference` algorithm whose type specifies the method to use and whose fields specify the configuration of the method.

As of 03/2021 the only implementation of `VariationalInference` available in `Turing.jl` is `ADVI`, which is applicable as long as the `Model` is differentiable with respect to the *variational parameters*.

By default, when calling `vi(m, advi)`, Turing uses a *mean-field* approximation with a multivariate normal as the base distribution. [Mean-field](https://en.wikipedia.org/wiki/Mean-field_theory) as borrowed from mean-field theory in statistical physics refers here to the fact that it is assumed all the latent variables are *independent*. This is standard approach in ADVI (see [Automatic Differentiation Variational Inference (2016)](https://arxiv.org/abs/1603.00788)).

In the mean-field approximation, the parameters of the variational distribution are the mean and variance for each latent variable.

In [26]:
print(@doc(Variational.ADVI))

```julia
struct ADVI{AD} <: AdvancedVI.VariationalInference{AD}
```

Automatic Differentiation Variational Inference (ADVI) with automatic differentiation backend `AD`.

# Fields

  * `samples_per_step::Int64`

    Number of samples used to estimate the ELBO in each optimization step.
  * `max_iters::Int64`

    Maximum number of gradient steps.


In [25]:
print(@doc(Variational.meanfield))

```
meanfield([rng, ]model::Model)
```

Creates a mean-field approximation with multivariate normal as underlying distribution.


In [29]:
# ADVI
advi = ADVI(10, 1000)
q = vi(m, advi);

┌ Info: [ADVI] Should only be seen once: optimizer created for θ
│   objectid(θ) = 7240786473657583843
└ @ AdvancedVI /home/jovyan/.julia/packages/AdvancedVI/8ttTK/src/AdvancedVI.jl:199
[ADVI] Optimizing...100% Time: 0:00:35


For such a small problem Turing's `NUTS` sampler is nearly as efficient as ADVI.

This is *not* the case in general. For very complex models `ADVI` produces very reasonable results in a much shorter time than `NUTS`.

One significant advantage of using `vi` is that we can sample from the resulting approximate posterior `q` with ease. In fact, the result of the `vi` call is a `TransformedDistribution` from Bijectors.jl, and it implements the Distributions.jl interface for a `Distribution`.

# Deep universal probabilistic programming and inference compilation

# Examples